# Add SpatialData

- Convert the cohort of fovs and labels into several spatialdata objets, one `spatialdata` object per FOV.
- Add metadata to the `spatialdata` objects and save them to LaminDB as artifacts.

## Setup


In [ ]:
import bionty as bt
import lamindb as ln
from distributed import Client, LocalCluster
from upath import UPath

import nbl

cluster = LocalCluster(n_workers=10)
client = Client(cluster)

In [ ]:
client

In [ ]:
ln.settings.sync_git_repo = "https://github.com/karadavis-lab/nbl.git"

In [ ]:
ln.track(project="Neuroblastoma")

In [ ]:
bt.settings.organism = "human"

In [ ]:
raw_data_dir = UPath("../../data/raw/nbl_cohort")

fov_dir = raw_data_dir / "images"
label_dir = raw_data_dir / "segmentation" / "labels"

In [ ]:
hu_sdatas = nbl.io.convert_cohort(
    fov_dir=fov_dir,
    label_dir=label_dir,
    filter_fovs=r"Hu-*",
    return_collection=False,
)

In [ ]:
nbl_sdatas = nbl.io.convert_cohort(
    fov_dir=fov_dir,
    label_dir=label_dir,
    filter_fovs=r"NBL-*",
    return_collection=False,
)

In [ ]:
nbl_sdata = nbl_sdatas["NBL-1-R5C8"]

# ln.Artifact.from_spatialdata(hu_sdata, key="control/Hu-Adrenal-Medulla-R5C12.zarr")

In [ ]:
clinical_data = ln.Artifact.filter(ln.Q(key__contains="clinical_data")).one().load()

In [ ]:
nbl_a = ln.Artifact.from_spatialdata(nbl_sdata, key="NBL-1-R5C8.zarr")

In [ ]:
clinical_data[clinical_data["FOV"] == "NBL-1-R5C8"]

In [ ]:
ln.curators.SpatialDataCurator

In [ ]:
for _, fov_row in clinical_data.iterrows():
    # fov = fov_row["FOV"]
    fov_name = fov_row["FOV"]
    fov_sdata = nbl_sdatas[fov_name]

    fov_sdata_artifact = ln.Artifact.from_spatialdata(fov_sdata, key=f"nbl_sdatas/{fov_name}.zarr")
    fov_sdata_artifact.save()

    for f in (features := ln.Feature.lookup()):
        match f:
            case ln.Feature():
                feature_name = f.name
            case _:
                continue
        feature_value = fov_row[feature_name]
        feature = ln.Feature.filter(ln.Q(name=feature_name)).one()
        fov_sdata_artifact.labels.add(records=ln.ULabel.filter(ln.Q(name=feature_value)), feature=feature)

    # nbl_a.labels.add(records = ln.ULabel(name=feature_name, value=feature_value), feature=f)

In [ ]:
ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas/NBL-7-R13C12.zarr")).latest_version().one().describe()

In [ ]:
ln.ULabel.filter(ln.Q(name=feature_value))

In [ ]:
ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas"))

In [ ]:
ln.Collection(ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas")))

In [ ]:
ln.Collection(artifacts=ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas")).latest_version(), key="nbl fovs")